# Important *Package*

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.metrics import accuracy_score
import random
from torch.utils import data
from tqdm import tqdm

import matplotlib.pyplot as plt
import time

# random seed
np.random.seed(11785)
torch.manual_seed(11785)

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
sys.version
print(cuda, sys.version)


True 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]


In [ ]:
# Check if cuda is available
cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))

Cuda = True with num_workers = 8


In [ ]:
from google.colab import drive
import json
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive
%ls

/content/gdrive/MyDrive
'Colab Notebooks'/          IPF/
 competitions/              kaggle/
'Computational Genomics'/   Recitation0D-Google_Colab_101_S21.gdoc
'Getting started.pdf'       Untitled.R
'HCS Assignment.gdoc'      'Untitled spreadsheet.gsheet'
 IDL/


# Torchvision

In [ ]:
# unzip
%cd /content/gdrive/My Drive/competitions/HW2P2-classification
%cp /content/gdrive/My\ Drive/competitions/HW2P2-classification/11785-spring2021-hw2p2s1-face-classification.zip  /content
%cd  /content
%ls
!unzip \*.zip

Streaming output truncated to the last 5000 lines.
  inflating: val_data/2348/0001_01.jpg  
  inflating: val_data/2348/0046_01.jpg  
  inflating: val_data/2349/0082_01.jpg  
  inflating: val_data/2349/0312_02.jpg  
  inflating: val_data/235/0008_02.jpg  
  inflating: val_data/235/0140_03.jpg  
  inflating: val_data/2350/0106_01.jpg  
  inflating: val_data/2350/0339_01.jpg  
  inflating: val_data/2351/0006_01.jpg  
  inflating: val_data/2351/0150_01.jpg  
  inflating: val_data/2352/0193_01.jpg  
  inflating: val_data/2352/0311_01.jpg  
  inflating: val_data/2353/0229_01.jpg  
  inflating: val_data/2353/0297_02.jpg  
  inflating: val_data/2354/0140_01.jpg  
  inflating: val_data/2354/0444_01.jpg  
  inflating: val_data/2355/0112_02.jpg  
  inflating: val_data/2355/0385_01.jpg  
  inflating: val_data/2356/0097_01.jpg  
  inflating: val_data/2356/0263_01.jpg  
  inflating: val_data/2357/0040_01.jpg  
  inflating: val_data/2357/0158_03.jpg  
  inflating: val_data/2358/0184_01.jpg  
  inflat

In [ ]:
transforms_train = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop((50,50)),
    torchvision.transforms.RandomHorizontalFlip(),
    # torchvision.transforms.RandomAffine(degrees=(-45,45), scale=(.9, 1.1), shear=0),
    torchvision.transforms.RandomRotation(degrees = (-30,30)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transforms_test = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root='/content/train_data', 
                                                 transform=transforms_train)
print(train_dataset.__len__(), len(train_dataset.classes))
dev_dataset = torchvision.datasets.ImageFolder(root='/content/val_data',
                                               transform=transforms_test)
print(dev_dataset.__len__(), len(dev_dataset.classes))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, 
                                               shuffle=True, num_workers=8)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=128, 
                                             shuffle=False, num_workers=8)

380638 4000
8000 4000


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
%pwd

'/content'

In [ ]:
class SimpleResidualBlock(nn.Module):
    def __init__(self, in_channel,out_channel, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)


        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride)
        self.relu = nn.ReLU()

    def forward(self, x):
        # print(np.shape(x))
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        shortcut = self.shortcut(x)
        # print(np.shape(out),np.shape(shortcut))
        out = self.relu(out + shortcut)

        return out


In [ ]:
# This has hard-coded hidden feature sizes.
# You can extend this to take in a list of hidden sizes as argument if you want.
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes,feat_dim = 1000):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1, padding=3, bias=False), # kernal 3 stride 1 & maxpool
            nn.BatchNorm2d(64),
            nn.ReLU(),

            SimpleResidualBlock(64,64), # *3
            SimpleResidualBlock(64,64),
            SimpleResidualBlock(64,64),

            SimpleResidualBlock(64,128,stride=2), # *4
            SimpleResidualBlock(128,128),
            SimpleResidualBlock(128,128),
            SimpleResidualBlock(128,128),

            SimpleResidualBlock(128,256,stride=2),   # *6
            SimpleResidualBlock(256,256),
            SimpleResidualBlock(256,256),
            SimpleResidualBlock(256,256),
            SimpleResidualBlock(256,256),
            SimpleResidualBlock(256,256),

            SimpleResidualBlock(256,512,stride=2), # *3
            SimpleResidualBlock(512,512),
            SimpleResidualBlock(512,512),


            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Dropout2d(p=0.5,inplace=True),
            nn.Flatten(), # the above ends up with batch_size x 64 x 1 x 1, flatten to batch_size x 64
        )
        self.linear = nn.Linear(512, feat_dim)
        self.relu = nn.ReLU(inplace=True)
        self.linear_output = nn.Linear(512,num_classes)
    def forward(self, x, return_embedding=False):
        embedding = self.layers(x)
        embedding_out = self.relu(self.linear(embedding))
        output = self.linear_output(embedding)
        if return_embedding:
            return embedding_out,output
        else:
            return output



In [ ]:
in_features = 3 # RGB channels
num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = ClassificationNetwork(in_features, num_classes)
network = network.to(device)


def reset_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant(m.weight, 1)
                nn.init.constant(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal(m.weight, std=0.001)
                if m.bias is not None:
                    nn.init.constant(m.bias, 0)


network.apply(reset_params)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


ClassificationNetwork(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): SimpleResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Identity()
      (relu): ReLU()
    )
    (4): SimpleResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3

In [ ]:
numEpochs = 40

learningRate = 0.01
weightDecay = 5e-5


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)# Sheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=5)


In [ ]:
%cd /content/gdrive/My Drive/competitions/HW2P2-classification

# Train 
best_accuracy = -1
accuracy_val = []
# network = torch.load('./model_classification_34_kaiming.pt')


# Validate
network.eval()
num_correct = 0
avg_loss_val = 0
with torch.no_grad():
    for batch_num, (x, y) in tqdm(enumerate(dev_dataloader),position=0, leave=True):
        x, y = x.to(device), y.to(device)
        outputs = network(x)

        loss = criterion(outputs, y.long())
        avg_loss_val += loss.item()

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
    best_accuracy = num_correct / len(dev_dataset)
    avg_loss_validation = avg_loss_val / len(dev_dataloader)
    print('accuracy',best_accuracy,'loss',avg_loss_validation)


for epoch in range(numEpochs):

    # Train
    network.train()
    avg_loss = 0.0
    train_accuracy = 0
    num_size = 0
    for batch_num, (x, y) in tqdm(enumerate(train_dataloader),position=0, leave=True):
        optimizer.zero_grad()

        x, y = x.to(device), y.to(device)

        outputs = network(x)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()

        _,pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        train_accuracy += torch.sum(torch.eq(pred_labels, y)).item()
        num_size += len(y)

        if batch_num % 100 == 99:
            print('Epoch: {}\tAvg-Loss: {:.4f}\tAccuract: {:.4f}'.format(epoch+1, avg_loss/100,train_accuracy/num_size))
            avg_loss = 0.0

    # Validate
    network.eval()
    num_correct = 0
    avg_loss_val = 0
    with torch.no_grad():
        for batch_num, (x, y) in tqdm(enumerate(dev_dataloader),position=0, leave=True):
            x, y = x.to(device), y.to(device)
            outputs = network(x)

            loss = criterion(outputs, y.long())
            avg_loss_val += loss.item()

            num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
        accuracy = num_correct / len(dev_dataset)
        accuracy_val.append(accuracy)
        avg_loss_validation = avg_loss_val / len(dev_dataloader)

        print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, accuracy))
        print('Epoch: {}, Validation loss: {:.2f}'.format(epoch, avg_loss_validation))
        scheduler.step(avg_loss_validation)
        print('Epoch {}, lr {}'.format(
              epoch, optimizer.param_groups[0]['lr']))

    if accuracy > best_accuracy:
      best_accuracy = accuracy
      print('val accuracy',accuracy_val)
      torch.save(network, 'mode_1.pt')


/content/gdrive/My Drive/competitions/HW2P2-classification


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
63it [00:12,  5.12it/s]


accuracy 0.798375 loss 1.0100465133076622


100it [00:28,  3.60it/s]

Epoch: 1	Avg-Loss: 0.9037	Accuract: 0.7832


...

900it [04:10,  3.59it/s]

Epoch: 12	Avg-Loss: 0.7489	Accuract: 0.8171


1000it [04:38,  3.59it/s]

Epoch: 12	Avg-Loss: 0.7140	Accuract: 0.8174


1052it [04:53,  3.59it/s]